# Compute average storage by month/day and station

### Import Python tools and Jupyter configuration

In [34]:
%load_ext lab_black

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [74]:
import pandas as pd
import altair as alt
import altair_latimes as lat
import datetime as dt

In [75]:
alt.themes.register("latimes", lat.theme)
alt.themes.enable("latimes")
import altair as alt
import altair_latimes as lat

In [89]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
alt.data_transformers.disable_max_rows()
pd.options.display.max_colwidth = None

In [90]:
today = dt.datetime.today().strftime("%Y-%m-%d")

### Read storage levels data from each reservoir

In [112]:
df = pd.read_json("data/processed/levels/historical_levels_2021-06-27.json")

In [113]:
df = df[(df["storage_acrefeet"] != -9999) & (df["monthday"] != "02-29")]

In [93]:
df.head(5)

,stationid,date,storage_acrefeet,year,quarter,day,month,weekday,monthname,monthday,date_str
0,SNL,1991-01-01,491232,1991,1,1,1,1,January,01-01,1991-01-01
1,SNL,1991-01-02,495043,1991,1,2,1,2,January,01-02,1991-01-02
2,SNL,1991-01-03,502621,1991,1,3,1,3,January,01-03,1991-01-03
3,SNL,1991-01-04,512765,1991,1,4,1,4,January,01-04,1991-01-04
4,SNL,1991-01-05,520760,1991,1,5,1,5,January,01-05,1991-01-05


In [94]:
avgs = (
    df.groupby(["monthday", "stationid"])
    .agg({"storage_acrefeet": "mean"})
    .round(0)
    .reset_index()
)

In [95]:
avgs.rename(columns={"storage_acrefeet": "mean_storage_af"}, inplace=True)

In [104]:
alt.Chart(avgs).mark_area().encode(
    x="monthday:T", y="mean_storage_af:Q", facet=alt.Facet("stationid", columns=4)
).properties(width=200, height=200)

alt.Chart(...)

In [97]:
avgs.to_json(
    "data/processed/averages/mean_monthday_station.json", indent=2, orient="records"
)